In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

In [2]:
ratings = pd.read_csv(r"C:\Users\sjlal\NYP\Project\FinalData\ratings.csv")
credits = pd.read_csv(r"C:\Users\sjlal\NYP\Project\FinalData\credits.csv")
keywords = pd.read_csv(r"C:\Users\sjlal\NYP\Project\FinalData\keywords.csv")
movies_metadata = pd.read_csv(r"C:\Users\sjlal\NYP\Project\FinalData\movies_metadata.csv")
links = pd.read_csv(r"C:\Users\sjlal\NYP\Project\FinalData\links.csv")

# Data cleaning

In [3]:
print(ratings.shape)
print(credits.shape)
print(keywords.shape)
print(movies_metadata.shape)
print(links.shape)

(26024289, 4)
(45476, 3)
(46419, 2)
(45463, 24)
(45843, 3)


In [4]:
#movies_metadata['id'] = movies_metadata['id'].astype(int)

In [5]:
#ratings has no duplicate
ratings.loc[ratings.duplicated(subset=['userId','movieId'], keep=False),:]


,userId,movieId,rating,timestamp


In [6]:
links.drop_duplicates(subset = ['tmdbId'], keep = False, inplace = True)
keywords.drop_duplicates(subset = ['id'], keep = 'first', inplace = True)
credits.drop_duplicates(subset = ['id'], keep = 'first', inplace = True)

In [7]:
#credits[credits.duplicated()]
#credits[credits['id'] == 105045]
#credits = credits.drop_duplicates()
#keywords = keywords.drop_duplicates()
#movies_metadata = movies_metadata.drop_duplicates()

In [8]:
print(ratings.shape)
print(credits.shape)
print(keywords.shape)
print(movies_metadata.shape)
print(links.shape)

(26024289, 4)
(45432, 3)
(45432, 2)
(45463, 24)
(45565, 3)


In [9]:
movies_metadata[movies_metadata['id'].duplicated(keep=False)].sort_values('id')

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
33824,False,NaN,30000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",NaN,4912,tt0270288,en,Confessions of a Dangerous Mind,"Television made him famous, but his biggest hi...",...,2002-12-30,33013805,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some things are better left top secret.,Confessions of a Dangerous Mind,False,6.6,281
5865,False,NaN,30000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",NaN,4912,tt0270288,en,Confessions of a Dangerous Mind,"Television made him famous, but his biggest hi...",...,2002-12-30,33013805,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some things are better left top secret.,Confessions of a Dangerous Mind,False,6.6,281
9165,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,5511,tt0062229,fr,Le Samouraï,Hitman Jef Costello is a perfectionist who alw...,...,1967-10-25,39481,105.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,There is no solitude greater than that of the ...,Le Samouraï,False,7.9,187
7345,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,5511,tt0062229,fr,Le Samouraï,Hitman Jef Costello is a perfectionist who alw...,...,1967-10-25,39481,105.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,There is no solitude greater than that of the ...,Le Samouraï,False,7.9,187
44818,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,...,2000-07-08,68411275,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,144
4114,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,...,2000-07-08,68411275,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,143
24843,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://www.dealthemovie.com/,11115,tt0446676,en,Deal,As an ex-gambler teaches a hot-shot college ki...,...,2008-01-29,0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Deal,False,5.2,22
14012,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://www.dealthemovie.com/,11115,tt0446676,en,Deal,As an ex-gambler teaches a hot-shot college ki...,...,2008-01-29,0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Deal,False,5.2,22
44823,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.pokemon.com/us/movies/movie-pokemon...,12600,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",...,2001-07-06,28023563,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,False,5.7,82
5535,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.pokemon.com/us/movies/movie-pokemon...,12600,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",...,2001-07-06,28023563,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,False,5.7,82


In [10]:
#average popularity for duplicates
duplicate_df = movies_metadata[movies_metadata['id'].duplicated(keep=False)]['id']
duplicate_index = movies_metadata[movies_metadata['id'].duplicated(keep=False)].index
duplicate_id = movies_metadata[movies_metadata['id'].duplicated(keep=False)]['id'].values
duplicate_changes = movies_metadata[movies_metadata['id'].duplicated(keep=False)].groupby('id')['popularity'].mean()

#movies_metadata.loc[duplicate_index] = movies_metadata.loc[duplicate_index].groupby('id')['popularity'].mean()

In [11]:
#for testing
duplicate_index

Int64Index([  676,   838,   949,  1465,  2564,  4114,  4356,  5130,  5535,
             5710,  5865,  7345,  8068,  9165,  9327,  9576, 10419, 11155,
            11342, 12066, 13220, 13261, 13375, 13603, 13946, 14000, 14012,
            15074, 15702, 15765, 16167, 16764, 17229, 19889, 19924, 20842,
            20898, 21115, 21164, 21853, 22150, 23043, 23533, 24163, 24843,
            26624, 28859, 29373, 29999, 33182, 33741, 33824, 35795, 38868,
            40037, 40273, 44818, 44823, 45262],
           dtype='int64')

In [12]:
#for testing
duplicate_changes

id
4912       9.488449
5511       9.091288
10991      8.372487
11115      6.880365
12600      6.576205
13209      1.529419
14788      3.096777
15028      5.146899
18440      1.436085
22649      2.162944
23305      1.967992
25541      2.587911
42495      0.187901
69234      0.440181
77221      6.563931
84198      1.087176
97995      0.141558
99080      0.002362
105045     0.122178
109962    11.288857
110428     0.122039
119916     0.000018
132641     0.357734
141971     0.411949
152795     8.534039
159849     0.473322
168538     1.276602
265189    12.165685
298721     2.535419
Name: popularity, dtype: float64

In [13]:
for i in duplicate_index:
    x = duplicate_changes[duplicate_df[i]]
    movies_metadata.loc[[i],'popularity'] = x
#    print(movies_metadata.loc[[i],'popularity'])
#    print(duplicate_changes[duplicate_df[i]])

In [14]:
movies_metadata = movies_metadata.drop_duplicates()
print(movies_metadata.shape)

(45435, 24)


In [15]:
#there are 2 duplicates with different vote_count. There is no way to check which is correct 1 is randomly dropped
movies_metadata.drop_duplicates(subset = ['id'], keep = 'first', inplace = True)
print(movies_metadata.shape)

(45433, 24)


In [16]:
print(ratings.shape)
print(credits.shape)
print(keywords.shape)
print(movies_metadata.shape)
print(links.shape)

(26024289, 4)
(45432, 3)
(45432, 2)
(45433, 24)
(45565, 3)


# Combine into 1 dataframe

In [17]:
df_combined = pd.merge(left = ratings, right = links, right_on='movieId', left_on='movieId', how='left')

In [18]:
df_combined.shape

(26024289, 6)

In [19]:
df_combined1 = pd.merge(left = df_combined, right = movies_metadata, right_on='id', left_on='tmdbId', how='left')

In [20]:
df_combined1.shape

(26024289, 30)

In [21]:
#export file to excel to check
#movies_metadata[movies_metadata['id'].duplicated(keep=False)].sort_values('id').to_csv("check.csv")

In [22]:
df_combined2 = pd.merge(left = df_combined1, right=credits, right_on='id', left_on='tmdbId', how='left')

In [23]:
df_combined2.shape

(26024289, 33)

In [24]:
df_combined3 = pd.merge(left=df_combined2, right=keywords, right_on='id', left_on='tmdbId', how='left')

In [25]:
df_combined3.shape

(26024289, 35)

# Data visualising

In [26]:
#looking at which fields is not necessary
pd.set_option('display.max_columns', None)
df_combined3.columns.values.tolist()

['userId',
 'movieId',
 'rating',
 'timestamp',
 'imdbId',
 'tmdbId',
 'adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id_x',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count',
 'cast',
 'crew',
 'id_y',
 'id',
 'keywords']

In [27]:
#looking at voting score
#df_combined3.groupby(['vote_average']).count()

In [28]:
#looking at user ratings
#takes too long
#df_combined3.groupby(['movieId','userId']).mean()

# Data Massage

In [29]:
#drop useless columns
#fit into model
#see results

In [30]:
#unable to run, not enough pc resources
#df_combined3.to_csv("compiled.csv")

In [79]:
# only require user-item data
raw_data = df_combined3[['genres','id','title','userId','movieId', 'rating']]

In [80]:
#raw_data[raw_data.isna().any(axis=1)]
raw_data.dropna(inplace=True)

<ipython-input-80-81826bbcaa4f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data.dropna(inplace=True)


In [81]:
import ast
def get_genre(outterlist):
    outterlist = ast.literal_eval(outterlist)
    lst = []
    for a in outterlist:
        try:
            lst.append(a['name'])
        except IndexError:
            lst.append('')
    return '|'.join(lst)

In [82]:
raw_data = raw_data.sample(n=100000, replace=False, random_state=42)

In [83]:
#Complete massage
raw_data['id'] = raw_data['id'].astype(str)
raw_data['movieId'] = raw_data['movieId'].astype(str)
raw_data['genres_descp'] = raw_data['genres'].apply(get_genre)
raw_data = raw_data.loc[~raw_data["id"].str.contains('-')]
raw_data = raw_data.loc[raw_data['genres_descp'].str.len()>3]



In [84]:
# setting up data for model
df_user_le = pd.DataFrame(raw_data["userId"].tolist(), columns=['user'])
le = preprocessing.LabelEncoder()
df_user_le['user_le'] = le.fit_transform(df_user_le['user'])
df_mv_le = pd.DataFrame(raw_data["movieId"].tolist(), columns=['movie'])
df_mv_le['mv_le'] = le.fit_transform(df_mv_le['movie'])

In [85]:
user_ids = np.array(df_user_le['user_le'].tolist())
movie_ids = np.array(df_mv_le['mv_le'].tolist())
user_ratings = np.array(raw_data["rating"].tolist())
print ('num of users:',raw_data['userId'].nunique())
print ('num of movies:',raw_data['movieId'].nunique())
NumUsers = raw_data['userId'].nunique()
NumMovie = raw_data['movieId'].nunique()

num of users: 58437
num of movies: 9323


In [86]:
graph = tf.Graph()
n_movie = NumMovie
n_user = NumUsers
embedding_size = 30

lr = 0.0001
reg = 0.01

with graph.as_default():
    user = tf.compat.v1.placeholder(tf.int32, name="user_id") 
    movie = tf.compat.v1.placeholder(tf.int32, name="movie_id") 
    rating = tf.compat.v1.placeholder(tf.float32, name="rating") 

    movie_embedding = tf.Variable(tf.random.truncated_normal([n_movie, embedding_size], stddev=0.02, mean=0.) ,name="movie_embedding")
    user_embedding = tf.Variable(tf.random.truncated_normal([n_user, embedding_size], stddev=0.02, mean=0.) ,name="user_embedding")
    
    movie_bias_embedding = tf.Variable(tf.random.truncated_normal([n_movie], stddev=0.02, mean=0.) ,name="movie_bias_embedding")
    user_bias_embedding = tf.Variable(tf.random.truncated_normal([n_user], stddev=0.02, mean=0.) ,name="user_bias_embedding")
    
    
    global_bias = tf.Variable(tf.random.truncated_normal([], stddev=0.02, mean=0.) ,name="global_bias")
    
    u = tf.nn.embedding_lookup(user_embedding, user)
    m = tf.nn.embedding_lookup(movie_embedding, movie)
    
    u_bias = tf.nn.embedding_lookup(user_bias_embedding, user)
    m_bias = tf.nn.embedding_lookup(movie_bias_embedding, movie)
    

    predicted_rating = tf.reduce_sum(tf.multiply(u, m), 1) + u_bias + m_bias + global_bias

    rmse = tf.sqrt(tf.reduce_mean(tf.square(predicted_rating - rating))) # RMSE
    cost = tf.nn.l2_loss(predicted_rating - rating)
    regularization = reg * (tf.nn.l2_loss(movie_embedding) + tf.nn.l2_loss(user_embedding)
                            + tf.nn.l2_loss(movie_bias_embedding) + tf.nn.l2_loss(user_bias_embedding))
    
    loss = cost + regularization
    
    optimizer = tf.compat.v1.train.AdamOptimizer(lr).minimize(loss)

In [87]:
#running the model

batch_size = 5
n_epoch = 40


with tf.compat.v1.Session(graph=graph) as sess:
    tf.compat.v1.global_variables_initializer().run()
    for _ in range(n_epoch):
        for start in range(0, user_ratings.shape[0] - batch_size, batch_size):
            end = start + batch_size
            _, cost_value = sess.run([optimizer, rmse], feed_dict={user: user_ids[start:end],
                                                  movie: movie_ids[start: end],
                                                  rating: user_ratings[start: end]})

        print ("RMSE", cost_value)
    embeddings = movie_embedding.eval()

RMSE 1.7795953
RMSE 0.861062
RMSE 0.84002906
RMSE 0.8344326
RMSE 0.83163226
RMSE 0.83023244
RMSE 0.82955426
RMSE 0.82924944
RMSE 0.8291333
RMSE 0.8291085
RMSE 0.8291238
RMSE 0.8291538
RMSE 0.8291861
RMSE 0.8292154
RMSE 0.82924014
RMSE 0.8292599
RMSE 0.82927495
RMSE 0.8292863
RMSE 0.82929456
RMSE 0.8293009
RMSE 0.82930547
RMSE 0.8293088
RMSE 0.82931125
RMSE 0.829313
RMSE 0.82931423
RMSE 0.8293151
RMSE 0.8293157
RMSE 0.82931614
RMSE 0.8293165
RMSE 0.82931685
RMSE 0.829317
RMSE 0.8293171
RMSE 0.82931715
RMSE 0.8293172
RMSE 0.82931733
RMSE 0.82931733
RMSE 0.82931733
RMSE 0.82931745
RMSE 0.82931745
RMSE 0.82931745


In [ ]:
samplesize = 10000


while samplesize < 100000:
    raw_data = df_combined3[['genres','id','title','userId','movieId', 'rating']]
    raw_data.dropna(inplace=True)
    raw_data = raw_data.sample(n=100000, replace=False, random_state=42)
    raw_data['id'] = raw_data['id'].astype(str)
    raw_data['movieId'] = raw_data['movieId'].astype(str)
    raw_data['genres_descp'] = raw_data['genres'].apply(get_genre)
    raw_data = raw_data.loc[~raw_data["id"].str.contains('-')]
    raw_data = raw_data.loc[raw_data['genres_descp'].str.len()>3]
    df_user_le = pd.DataFrame(raw_data["userId"].tolist(), columns=['user'])
    le = preprocessing.LabelEncoder()
    df_user_le['user_le'] = le.fit_transform(df_user_le['user'])
    df_mv_le = pd.DataFrame(raw_data["movieId"].tolist(), columns=['movie'])
    df_mv_le['mv_le'] = le.fit_transform(df_mv_le['movie'])
    user_ids = np.array(df_user_le['user_le'].tolist())
    movie_ids = np.array(df_mv_le['mv_le'].tolist())
    user_ratings = np.array(raw_data["rating"].tolist())
    NumUsers = raw_data['userId'].nunique()
    NumMovie = raw_data['movieId'].nunique()
    graph = tf.Graph()
    n_movie = NumMovie
    n_user = NumUsers
    embedding_size = 30
    lr = 0.0001
    reg = 0.01
    with graph.as_default():
        user = tf.compat.v1.placeholder(tf.int32, name="user_id") 
        movie = tf.compat.v1.placeholder(tf.int32, name="movie_id") 
        rating = tf.compat.v1.placeholder(tf.float32, name="rating") 

        movie_embedding = tf.Variable(tf.random.truncated_normal([n_movie, embedding_size], stddev=0.02, mean=0.) ,name="movie_embedding")
        user_embedding = tf.Variable(tf.random.truncated_normal([n_user, embedding_size], stddev=0.02, mean=0.) ,name="user_embedding")

        movie_bias_embedding = tf.Variable(tf.random.truncated_normal([n_movie], stddev=0.02, mean=0.) ,name="movie_bias_embedding")
        user_bias_embedding = tf.Variable(tf.random.truncated_normal([n_user], stddev=0.02, mean=0.) ,name="user_bias_embedding")


        global_bias = tf.Variable(tf.random.truncated_normal([], stddev=0.02, mean=0.) ,name="global_bias")

        u = tf.nn.embedding_lookup(user_embedding, user)
        m = tf.nn.embedding_lookup(movie_embedding, movie)

        u_bias = tf.nn.embedding_lookup(user_bias_embedding, user)
        m_bias = tf.nn.embedding_lookup(movie_bias_embedding, movie)


        predicted_rating = tf.reduce_sum(tf.multiply(u, m), 1) + u_bias + m_bias + global_bias

        rmse = tf.sqrt(tf.reduce_mean(tf.square(predicted_rating - rating))) # RMSE
        cost = tf.nn.l2_loss(predicted_rating - rating)
        regularization = reg * (tf.nn.l2_loss(movie_embedding) + tf.nn.l2_loss(user_embedding)
                                + tf.nn.l2_loss(movie_bias_embedding) + tf.nn.l2_loss(user_bias_embedding))

        loss = cost + regularization

        optimizer = tf.compat.v1.train.AdamOptimizer(lr).minimize(loss)

    batch_size = 5
    n_epoch = 1


    with tf.compat.v1.Session(graph=graph) as sess:
        tf.compat.v1.global_variables_initializer().run()
        for _ in range(n_epoch):
            for start in range(0, user_ratings.shape[0] - batch_size, batch_size):
                end = start + batch_size
                _, cost_value = sess.run([optimizer, rmse], feed_dict={user: user_ids[start:end],
                                                      movie: movie_ids[start: end],
                                                      rating: user_ratings[start: end]})

            print ("RMSE", cost_value)
        embeddings = movie_embedding.eval()
    samplesize += 10000
    
    